# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [5]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,80.56,90,66,5.01,broken clouds
1,1,La Roche-Sur-Yon,FR,46.6667,-1.4333,50.32,87,75,8.05,broken clouds
2,2,Torbay,CA,47.6666,-52.7314,37.27,79,75,16.11,broken clouds
3,3,Saint-Philippe,RE,-21.3585,55.7679,67.59,75,34,9.15,light rain
4,4,Ahipara,NZ,-35.1667,173.1667,65.34,85,100,1.10,overcast clouds
5,5,Mocuba,MZ,-16.8392,36.9856,72.63,90,61,1.14,broken clouds
6,6,Mataura,NZ,-46.1927,168.8643,58.48,57,99,2.48,overcast clouds
7,7,Kapaa,US,22.0752,-159.3190,79.12,63,40,12.66,scattered clouds
8,8,Faanui,PF,-16.4833,-151.7500,78.46,77,98,1.36,light rain
9,9,Busselton,AU,-33.6500,115.3333,53.47,86,43,7.92,scattered clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [6]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [7]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,80.56,90,66,5.01,broken clouds
7,7,Kapaa,US,22.0752,-159.3190,79.12,63,40,12.66,scattered clouds
8,8,Faanui,PF,-16.4833,-151.7500,78.46,77,98,1.36,light rain
10,10,Jacqueville,CI,5.2052,-4.4146,80.53,78,11,10.54,few clouds
12,12,Cockburn Town,TC,21.4612,-71.1419,84.16,70,40,11.50,scattered clouds
13,13,Yeppoon,AU,-23.1333,150.7333,76.78,80,87,4.92,overcast clouds
18,18,Kahului,US,20.8947,-156.4700,77.97,83,75,12.66,broken clouds
21,21,Nueve De Julio,AR,-35.4444,-60.8831,85.96,31,2,7.00,clear sky
23,23,The Valley,AI,18.2170,-63.0578,84.06,78,20,14.97,few clouds
31,31,Atuona,PF,-9.8000,-139.0333,77.65,72,4,17.29,clear sky


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [8]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna(
    axis=0,
    how='any'
)

clean_travel_cities.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# Display sample data
clean_travel_cities.head(20)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,80.56,90,66,5.01,broken clouds
7,7,Kapaa,US,22.0752,-159.3190,79.12,63,40,12.66,scattered clouds
8,8,Faanui,PF,-16.4833,-151.7500,78.46,77,98,1.36,light rain
10,10,Jacqueville,CI,5.2052,-4.4146,80.53,78,11,10.54,few clouds
12,12,Cockburn Town,TC,21.4612,-71.1419,84.16,70,40,11.50,scattered clouds
13,13,Yeppoon,AU,-23.1333,150.7333,76.78,80,87,4.92,overcast clouds
18,18,Kahului,US,20.8947,-156.4700,77.97,83,75,12.66,broken clouds
21,21,Nueve De Julio,AR,-35.4444,-60.8831,85.96,31,2,7.00,clear sky
23,23,The Valley,AI,18.2170,-63.0578,84.06,78,20,14.97,few clouds
31,31,Atuona,PF,-9.8000,-139.0333,77.65,72,4,17.29,clear sky


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [12]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description","Lat", "Lng"]].copy()

# Display sample data
hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng
0,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518
7,Kapaa,US,79.12,scattered clouds,22.0752,-159.3190
8,Faanui,PF,78.46,light rain,-16.4833,-151.7500
10,Jacqueville,CI,80.53,few clouds,5.2052,-4.4146
12,Cockburn Town,TC,84.16,scattered clouds,21.4612,-71.1419
13,Yeppoon,AU,76.78,overcast clouds,-23.1333,150.7333
18,Kahului,US,77.97,broken clouds,20.8947,-156.4700
21,Nueve De Julio,AR,85.96,clear sky,-35.4444,-60.8831
23,The Valley,AI,84.06,few clouds,18.2170,-63.0578
31,Atuona,PF,77.65,clear sky,-9.8000,-139.0333


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [13]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,
7,Kapaa,US,79.12,scattered clouds,22.0752,-159.3190,
8,Faanui,PF,78.46,light rain,-16.4833,-151.7500,
10,Jacqueville,CI,80.53,few clouds,5.2052,-4.4146,
12,Cockburn Town,TC,84.16,scattered clouds,21.4612,-71.1419,
13,Yeppoon,AU,76.78,overcast clouds,-23.1333,150.7333,
18,Kahului,US,77.97,broken clouds,20.8947,-156.4700,
21,Nueve De Julio,AR,85.96,clear sky,-35.4444,-60.8831,
23,The Valley,AI,84.06,few clouds,18.2170,-63.0578,
31,Atuona,PF,77.65,clear sky,-9.8000,-139.0333,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [14]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [15]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
# Display sample data
hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
7,Kapaa,US,79.12,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
8,Faanui,PF,78.46,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
10,Jacqueville,CI,80.53,few clouds,5.2052,-4.4146,Hotel Le Martin Pecheur
12,Cockburn Town,TC,84.16,scattered clouds,21.4612,-71.1419,Osprey Beach Hotel
13,Yeppoon,AU,76.78,overcast clouds,-23.1333,150.7333,Villa Mar Colina
18,Kahului,US,77.97,broken clouds,20.8947,-156.4700,Maui Seaside Hotel
21,Nueve De Julio,AR,85.96,clear sky,-35.4444,-60.8831,Grand Hotel Libertad - GHL
23,The Valley,AI,84.06,few clouds,18.2170,-63.0578,CeBlue Villas & Beach Resort
31,Atuona,PF,77.65,clear sky,-9.8000,-139.0333,Villa Enata


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [17]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != '']

# Display sample data
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
7,Kapaa,US,79.12,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
8,Faanui,PF,78.46,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
10,Jacqueville,CI,80.53,few clouds,5.2052,-4.4146,Hotel Le Martin Pecheur
12,Cockburn Town,TC,84.16,scattered clouds,21.4612,-71.1419,Osprey Beach Hotel
13,Yeppoon,AU,76.78,overcast clouds,-23.1333,150.7333,Villa Mar Colina
18,Kahului,US,77.97,broken clouds,20.8947,-156.4700,Maui Seaside Hotel
21,Nueve De Julio,AR,85.96,clear sky,-35.4444,-60.8831,Grand Hotel Libertad - GHL
23,The Valley,AI,84.06,few clouds,18.2170,-63.0578,CeBlue Villas & Beach Resort
31,Atuona,PF,77.65,clear sky,-9.8000,-139.0333,Villa Enata


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [19]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [21]:
import csv

# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file,index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [22]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [23]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Finch Bay Galapagos Hotel</dd>\n<dt>City</dt><dd>Puerto Ayora</dd>\n<dt>Country</dt><dd>EC</dd>\n<dt>Current Weather</dt><dd>broken clouds and 80.56 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Sheraton Kauai Coconut Beach Resort</dd>\n<dt>City</dt><dd>Kapaa</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 79.12 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Bora-Bora Pearl Beach Resort & Spa</dd>\n<dt>City</dt><dd>Faanui</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Current Weather</dt><dd>light rain and 78.46 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Le Martin Pecheur</dd>\n<dt>City</dt><dd>Jacqueville</dd>\n<dt>Country</dt><dd>CI</dd>\n<dt>Current Weather</dt><dd>few clouds and 80.53 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Osprey Beach Hotel</dd>\n<dt>City</dt><dd>Cockburn Town</dd>\n<dt>Country</dt><dd>TC</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 84.16 °F</dd>\n</dl>\n'

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [24]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
0,-0.7393,-90.3518
7,22.0752,-159.3190
8,-16.4833,-151.7500
10,5.2052,-4.4146
12,21.4612,-71.1419
13,-23.1333,150.7333
18,20.8947,-156.4700
21,-35.4444,-60.8831
23,18.2170,-63.0578
31,-9.8000,-139.0333


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [26]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(20.0, 10.0), zoom_level=1.75)
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))